# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [5]:
# Your code here
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [6]:
# Your code here
#Control_Column ~ C(factor_col1) + factor_col2 + C(factor_col3) + ... + X
formula = 'len ~ C(supp) + dose'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df           F        PR(>F)
C(supp)    205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [ ]:
# Your comment here
#Both dosage and and the supplement are likely to have an impact on tooth length. Dosage is likely
#to have the strongest effect.

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [14]:
# Your code here
df_oj = df.loc[df.supp == 'OJ']
df_vc = df.loc[df.supp == 'VC']
from flatiron_stats import welch_t, welch_df, p_value_welch_ttest

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [16]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
result = p_value_welch_ttest(df_oj.len, df_vc.len, two_sided=True)
print(result)

0.06063450788093383


## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [17]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
formula2 = 'len ~ C(supp)'
lm2 = ols(formula2, df).fit()
table2 = sm.stats.anova_lm(lm2, typ=2)
print(table2)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [22]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)
    
from itertools import combinations

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [35]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)
groups = [('OJ', 0.5),
('OJ', 1.0),
('OJ', 2.0),
('VC', 0.5),
('VC', 1.0),
('VC', 2.0)]
combos = list(combinations(groups, 2))
combos

    

[(('OJ', 0.5), ('OJ', 1.0)),
 (('OJ', 0.5), ('OJ', 2.0)),
 (('OJ', 0.5), ('VC', 0.5)),
 (('OJ', 0.5), ('VC', 1.0)),
 (('OJ', 0.5), ('VC', 2.0)),
 (('OJ', 1.0), ('OJ', 2.0)),
 (('OJ', 1.0), ('VC', 0.5)),
 (('OJ', 1.0), ('VC', 1.0)),
 (('OJ', 1.0), ('VC', 2.0)),
 (('OJ', 2.0), ('VC', 0.5)),
 (('OJ', 2.0), ('VC', 1.0)),
 (('OJ', 2.0), ('VC', 2.0)),
 (('VC', 0.5), ('VC', 1.0)),
 (('VC', 0.5), ('VC', 2.0)),
 (('VC', 1.0), ('VC', 2.0))]

In [30]:
df.loc[(df.supp == 'OJ') & (df.dose == 0.5), 'len']

30    15.2
31    21.5
32    17.6
33     9.7
34    14.5
35    10.0
36     8.2
37     9.4
38    16.5
39     9.7
Name: len, dtype: float64

In [36]:
df.loc[df.supp == 'OJ', 'len']
pvals = []
for combo in combos:
    pval = p_value_welch_ttest(df.loc[(df.supp == combo[0][0]) & (df.dose == combo[0][1]),'len'], 
                               df.loc[(df.supp == combo[1][0]) & (df.dose == combo[1][1]), 'len'], 
                               two_sided=True)
    pvals.append(pval)
    
pvals

[8.784919055160323e-05,
 1.323783877626994e-06,
 0.00635860676409683,
 0.04601033257637566,
 7.196253523966689e-06,
 0.03919514204624397,
 3.655206737285255e-08,
 0.0010383758722998238,
 0.09652612338267019,
 1.3621326289126046e-11,
 2.3610742028168374e-07,
 0.9638515887233756,
 6.811017703167721e-07,
 4.681577414622495e-08,
 9.155603056631989e-05]

In [37]:
two_test_pvals = dict(zip(combos, pvals))
two_test_pvals

{(('OJ', 0.5), ('OJ', 1.0)): 8.784919055160323e-05,
 (('OJ', 0.5), ('OJ', 2.0)): 1.323783877626994e-06,
 (('OJ', 0.5), ('VC', 0.5)): 0.00635860676409683,
 (('OJ', 0.5), ('VC', 1.0)): 0.04601033257637566,
 (('OJ', 0.5), ('VC', 2.0)): 7.196253523966689e-06,
 (('OJ', 1.0), ('OJ', 2.0)): 0.03919514204624397,
 (('OJ', 1.0), ('VC', 0.5)): 3.655206737285255e-08,
 (('OJ', 1.0), ('VC', 1.0)): 0.0010383758722998238,
 (('OJ', 1.0), ('VC', 2.0)): 0.09652612338267019,
 (('OJ', 2.0), ('VC', 0.5)): 1.3621326289126046e-11,
 (('OJ', 2.0), ('VC', 1.0)): 2.3610742028168374e-07,
 (('OJ', 2.0), ('VC', 2.0)): 0.9638515887233756,
 (('VC', 0.5), ('VC', 1.0)): 6.811017703167721e-07,
 (('VC', 0.5), ('VC', 2.0)): 4.681577414622495e-08,
 (('VC', 1.0), ('VC', 2.0)): 9.155603056631989e-05}

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.